In [1]:
import pymannkendall as mk
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#conda install -c conda-forge pymannkendall

In [29]:
#uploaded.keys()
in_file = 'tas_CanESM5_historical.nc'                      ## Input file
Data = xr.open_dataset(in_file)                            ## read in the data
Data=Data['tas'].groupby('time.year').mean('time')         ## Group data into yearly means
#Data

In [5]:
slope_val = np.zeros((len(Data.lat.values),len(Data.lon.values)))
p_value = np.zeros((len(Data.lat.values),len(Data.lon.values)))
#output = []
for i in np.arange(len(Data.lat.values)):
    for j in np.arange(len(Data.lon.values)):
        
        try:
            slope_val[i,j] = mk.original_test(Data[:,i,j]).slope  ## trend,h,p,z,tau,s,var_s,slope,intercept = mk.original_test(x,0.05)
            p_value[i,j] = mk.original_test(Data[:,i,j]).p
        except:
            slope_val[i,j] = np.nan
            p_value[i,j] = np.nan
            
        #output.append(slope_val)

In [41]:
## Define data as Xarray dataset and save as netcdf

output1=xr.DataArray(slope_val, dims=('lat', 'lon'), coords={'lat':Data.lat, 'lon':Data.lon}, attrs=dict(description="slope.", units="degC year-1"),)
data1 = output1.rename("trend")
output2=xr.DataArray(p_value, dims=('lat', 'lon'), coords={'lat':Data.lat, 'lon':Data.lon}, attrs=dict(description="significance",),)
data2 = output2.rename("p_val")

## Save Data as netcdf

data.to_netcdf('tas_CanESM5_historical_trend.nc', mode='w')
data.to_netcdf('tas_CanESM5_historical_pval.nc', mode='w')